In [1]:
%run ../commons/imports.ipynb
%run ../commons/helper_db.ipynb
%run ../commons/helper_contents.ipynb

In [2]:
MAIN_FOLDER = main_folder("tormarket")

create_database('blp_dataset')
create_table('products_tmp_2')

In [7]:
original_images = []

folders = [x for x in Path(MAIN_FOLDER).iterdir() if x.is_dir()]

for fo in folders:
    folder = Path(f"{fo}\products")
    folder_image = Path(f"{fo}")

    if folder.exists():
        for file in os.listdir(folder):
            page_content = BeautifulSoup(open(Path(f"{folder}\{file}"), encoding='utf8').read())
            
            if page_content is not None:
                ## GET DESCRIPTION
                description = get_content(page_content.find("pre"))

                ## GET TITLE
                raw_title = page_content.find("div", attrs={"id": "product-title"})
                title = get_content(raw_title)

                ## GET SELLER
                seller = ""
                raw_seller = page_content.find_all("a", href=lambda href: href and "/users/" in href)

                if raw_seller:
                    seller = get_content(raw_seller[0])

                
                if description != "" or title != "":
                    # Insert product into database
                    query = "INSERT INTO products_tmp_2 (name, description, market_name, seller_name, illegal) VALUES (%s, %s, %s, %s, %s)"
                    values = (title, description, "Tor Market", seller, "t")

                    cursor.execute(query, values)
                    conn.commit()

                    product_id = cursor.lastrowid
                    
                    ## GET IMAGE
                    img_content = page_content.find_all("img", src=lambda src: src and "/uploaded/" in src)
                    name = ""
                    
                    if img_content:
                        raw_name = img_content[0]
                        
                        if raw_name is not None:
                            try:
                                name = raw_name["src"]
                            except IndexError:
                                print(f"Image {product_id} has no src")
                                update_image(product_id, "no_image")

                        if name is not "":
                            name = name.replace("?", "_")
                            url_image = Path(f"{folder_image}\{name}")
                            ver = 0

                            if not url_image.exists():
                                name = name.replace("thumb", "medium")
                                url_image = Path(f"{folder_image}\{name}")
                                if url_image.exists(): ver = 1
                                
                            if not url_image.exists() and ver == 0:
                                name = name.replace("thumb", "original")
                                url_image = Path(f"{folder_image}\{name}")
                                if url_image.exists(): ver = 1
                            
                            if not url_image.exists() and ver == 0:
                                    name = name.replace("medium", "thumb")
                                    url_image = Path(f"{folder_image}\{name}")
                                    if url_image.exists(): ver = 1
                            
                            if not url_image.exists() and ver == 0:
                                    name = name.replace("medium", "original")
                                    url_image = Path(f"{folder_image}\{name}")
                                    if url_image.exists(): ver = 1
                            
                            if not url_image.exists() and ver == 0:
                                    name = name.replace("original", "thumb")
                                    url_image = Path(f"{folder_image}\{name}")
                                    if url_image.exists(): ver = 1
                            
                            if not url_image.exists() and ver == 0:
                                    name = name.replace("original", "medium")
                                    url_image = Path(f"{folder_image}\{name}")
                                    if url_image.exists(): ver = 1

                            if url_image.exists():
                                try:
                                    # copy images to new directory, renaming with product id
                                    if url_image.exists():
                                        print(f"Image {product_id} exists... creating")

                                        dst_dir = Path(f"D:\images\p\{product_id}.jpg")
                                        copyfile(url_image, dst_dir)
                                    else:
                                        print(f"Image {product_id} does not exists")
                                        update_image(product_id, "no_image")
                                except:
                                    print(f"Image {product_id} has error. Except")
                                    update_image(product_id, "no_image")
                            else:
                                print(f"Image {product_id} does not exists")
                                update_image(product_id, "no_image")
                        else:
                            print(f"Image {product_id} no exists")
                            update_image(product_id, "no_image")
                        
                    else:
                        print(f"Product {product_id} has no image")
                        update_image(product_id, "no_image")

Image 1 exists... creating
Image 2 exists... creating
Image 3 exists... creating
Image 4 exists... creating
Image 5 exists... creating
Image 6 exists... creating
Image 7 exists... creating
Image 8 exists... creating
Image 9 exists... creating
Image 10 exists... creating
Image 11 exists... creating
Image 12 exists... creating
Image 13 exists... creating
Image 14 exists... creating
Image 15 exists... creating
Image 16 exists... creating
Image 17 exists... creating
Image 18 exists... creating
Image 19 exists... creating
Image 20 exists... creating
Image 21 exists... creating
Image 22 exists... creating
Image 23 exists... creating
Image 24 exists... creating
Image 25 exists... creating
Image 26 exists... creating
Image 27 exists... creating
Image 28 exists... creating
Image 29 exists... creating
Image 30 exists... creating
Image 31 exists... creating
Image 32 exists... creating
Image 33 exists... creating
Image 34 exists... creating
Image 35 exists... creating
Image 36 exists... creating
I

Image 291 exists... creating
Product 292 has no image
Image 293 exists... creating
Image 294 exists... creating
Image 295 exists... creating
Image 296 exists... creating
Image 297 exists... creating
Image 298 exists... creating
Image 299 exists... creating
Image 300 exists... creating
Product 301 has no image
Image 302 exists... creating
Image 303 exists... creating
Image 304 exists... creating
Image 305 exists... creating
Product 306 has no image
Image 307 exists... creating
Image 308 exists... creating
Image 309 exists... creating
Image 310 exists... creating
Image 311 exists... creating
Image 312 exists... creating
Image 313 exists... creating
Product 314 has no image
Image 315 exists... creating
Image 316 exists... creating
Image 317 exists... creating
Image 318 exists... creating
Image 319 exists... creating
Image 320 exists... creating
Image 321 exists... creating
Image 322 exists... creating
Product 323 has no image
Product 324 has no image
Image 325 exists... creating
Image 326

Image 581 exists... creating
Image 582 exists... creating
Image 583 exists... creating
Image 584 exists... creating
Image 585 exists... creating
Image 586 exists... creating
Image 587 exists... creating
Image 588 exists... creating
Image 589 exists... creating
Image 590 exists... creating
Image 591 exists... creating
Image 592 exists... creating
Image 593 exists... creating
Image 594 exists... creating
Product 595 has no image
Image 596 exists... creating
Image 597 exists... creating
Image 598 exists... creating
Image 599 exists... creating
Image 600 exists... creating
Image 601 exists... creating
Image 602 does not exists
Image 603 exists... creating
Image 604 exists... creating
Image 605 exists... creating
Image 606 exists... creating
Image 607 exists... creating
Image 608 exists... creating
Image 609 exists... creating
Image 610 exists... creating
Image 611 exists... creating
Image 612 does not exists
Image 613 exists... creating
Image 614 exists... creating
Product 615 has no image

Image 867 exists... creating
Image 868 exists... creating
Image 869 exists... creating
Image 870 exists... creating
Product 871 has no image
Image 872 exists... creating
Image 873 exists... creating
Image 874 exists... creating
Image 875 exists... creating
Image 876 exists... creating
Image 877 exists... creating
Image 878 exists... creating
Image 879 exists... creating
Image 880 exists... creating
Image 881 exists... creating
Image 882 exists... creating
Image 883 exists... creating
Image 884 exists... creating
Image 885 exists... creating
Image 886 exists... creating
Image 887 exists... creating
Image 888 exists... creating
Image 889 exists... creating
Image 890 exists... creating
Image 891 exists... creating
Image 892 exists... creating
Image 893 exists... creating
Image 894 exists... creating
Image 895 exists... creating
Image 896 exists... creating
Image 897 exists... creating
Image 898 exists... creating
Image 899 exists... creating
Image 900 exists... creating
Image 901 exists..

Image 1151 exists... creating
Image 1152 exists... creating
Image 1153 exists... creating
Image 1154 exists... creating
Image 1155 exists... creating
Image 1156 exists... creating
Image 1157 exists... creating
Image 1158 exists... creating
Image 1159 exists... creating
Image 1160 exists... creating
Image 1161 exists... creating
Image 1162 exists... creating
Image 1163 exists... creating
Image 1164 exists... creating
Product 1165 has no image
Image 1166 exists... creating
Image 1167 exists... creating
Image 1168 exists... creating
Image 1169 exists... creating
Image 1170 exists... creating
Image 1171 exists... creating
Image 1172 exists... creating
Image 1173 does not exists
Image 1174 exists... creating
Image 1175 exists... creating
Image 1176 exists... creating
Image 1177 exists... creating
Image 1178 exists... creating
Product 1179 has no image
Image 1180 exists... creating
Image 1181 exists... creating
Image 1182 exists... creating
Image 1183 exists... creating
Image 1184 exists... 

Image 1430 does not exists
Image 1431 exists... creating
Image 1432 exists... creating
Image 1433 exists... creating
Image 1434 exists... creating
Image 1435 does not exists
Image 1436 exists... creating
Image 1437 exists... creating
Product 1438 has no image
Image 1439 exists... creating
Image 1440 exists... creating
Image 1441 exists... creating
Image 1442 exists... creating
Image 1443 exists... creating
Image 1444 exists... creating
Image 1445 does not exists
Image 1446 exists... creating
Image 1447 does not exists
Image 1448 exists... creating
Image 1449 exists... creating
Image 1450 exists... creating
Product 1451 has no image
Image 1452 exists... creating
Image 1453 does not exists
Image 1454 exists... creating
Image 1455 exists... creating
Image 1456 exists... creating
Image 1457 exists... creating
Image 1458 does not exists
Image 1459 exists... creating
Image 1460 exists... creating
Image 1461 exists... creating
Image 1462 exists... creating
Image 1463 exists... creating
Image 

Image 1710 exists... creating
Image 1711 exists... creating
Image 1712 does not exists
Image 1713 exists... creating
Image 1714 does not exists
Image 1715 does not exists
Image 1716 exists... creating
Image 1717 exists... creating
Image 1718 does not exists
Image 1719 exists... creating
Image 1720 exists... creating
Image 1721 exists... creating
Image 1722 exists... creating
Image 1723 exists... creating
Image 1724 exists... creating
Image 1725 exists... creating
Image 1726 exists... creating
Image 1727 exists... creating
Image 1728 exists... creating
Image 1729 exists... creating
Image 1730 exists... creating
Image 1731 exists... creating
Image 1732 exists... creating
Image 1733 exists... creating
Image 1734 exists... creating
Image 1735 does not exists
Image 1736 exists... creating
Image 1737 exists... creating
Image 1738 exists... creating
Image 1739 exists... creating
Image 1740 exists... creating
Image 1741 exists... creating
Product 1742 has no image
Image 1743 exists... creating